In [ ]:
##################################################################
# batch를 돌릴때 가장 먼저 실행합니다.
# 아래 설정 데이터들을 묶어 batch.config.json 파일로 저장합니다. (노트북이 있는 폴더에)
# 노트북 숫자 순서대로 실행
# 나눠서 각자 배치를 돌릴때는 아래 두 파일만 실행합니다.
#       (1) 00_config 의 정보를 본인에게 맞게 수정한 후 실행  
#       (2) 05_call_matches_timelines_api 를 실행 (40명분 데이터, 약 4시간반 소요 예상. 전원/네트워크 주의)
# 사전에 필요한 작업은 노트북 하단 주석을 참고해주세요.
##################################################################

In [1]:
# batch 돌릴때마다 이름을 변경해야한다. 
# 해당 batch의 파일을 저장하는 폴더명, 파일명으로 쓰일것이다.
batch_name = '181106'

# test 인지 아닌지 설정
# True이면 로컬에, False이면 AWS에 데이터를 쌓는다.
is_test = False

# LOL Season 
lol_season = 11

# LOL api token
# 나눠서 각자 배치를 돌릴 경우에는 각자 발급받은 24시간용 토큰키를 넣어준다.
# 불변키 문자열 : RGAPI-c07b29ed-3500-4800-xxxx-xxxxxxxxxxxx
token_key = "RGAPI-c07b29ed-3500-4800-xxxx-xxxxxxxxxxxx"

# 나눠서 각자 배치를 돌릴때 필요한 값들이다. 
# chunk count : 플레이어를 몇개의 청크로 나눌것인가
# my chunk idx : 내가 돌릴 플레이어 청크 인덱스 (1:권다정, 2:이승하, 3:임종운, 4:정현주, 5:한은구)
chunk_count = 5
my_chunk_idx = 1

# temp_file_path : 임시파일 저장소
# local로만 진행. 아직 사용하는 곳이 없다.
temp_file_path = '/Users/dajeongkwon/Documents/_TrollCamp/_data/temp'

# local file path : RAW 데이터들이 저장될 기본 장소
# 로컬 테스트시에 본인 디렉토리로 변경한다.
file_path = '/Users/dajeongkwon/Documents/_TrollCamp/_data'

# db information
aws_db_config = { 
    'user': 'xxxx-user',
    'password': 'xxxx-password',
    'host': 'xxxx.yyyyyyyy.ap-northeast-2.rds.amazonaws.com',
    'database': 'xxxx-database',
    'raise_on_warnings': True
}
local_db_config = {
    'user': 'xxxx-user',
    'password': 'xxxx-password',
    'host': '127.0.0.1',
    'database': 'xxxx-database',
    'raise_on_warnings': True
}

# S3 information
s3_bucket_name = 'xxxx-bucket_name'
s3_access_key_id = 'XXXX-access_key_id'
s3_secret_access_key = 'XXXX-secret_access_key'


#################


import json

db_config = local_db_config if is_test else aws_db_config

config_dict = {
    'batch_name': batch_name,
    'is_test': is_test,
    'lol_season': lol_season,
    'token_key': token_key,
    'chunk_count': chunk_count,
    'my_chunk_idx': my_chunk_idx,
    'temp_file_path': temp_file_path,
    'file_path': file_path,
    'db_config': db_config,
    's3_bucket_name': s3_bucket_name,
    's3_access_key_id': s3_access_key_id,
    's3_secret_access_key': s3_secret_access_key
}

with open('batch_config.json', 'w') as f:
    json.dump(config_dict, f)
print("new file : batch_config.json")

new file : batch_config.json


In [4]:
# (boto3 설치가 필요함) - 맥 기준 명령어

# $ sudo pip3 install boto3

In [3]:
# (mysql-connector 설치가 필요함) - 맥 기준 명령어

# $ git clone https://github.com/mysql/mysql-connector-python.git
# $ cd mysql-connector-python
# $ python3 ./setup.py build
# $ sudo python3 ./setup.py install

In [5]:
# AWS RDS 접근하려면 내 아이피를 보안그룹에 등록해야한다.
# AWS console 로그인 (trello 참고)
# RDS > 인스턴스 > oinglol
# 스크롤 중간쯤 내리면 "세부 정보"  >  "보안 및 네트워크" > "보안 그룹" 이 있다. 클릭
# 아래쪽 인바운드 > 편집 > 규칙추가 (내 IP)